# Example 3: Home Advantage Analysis

Tasks

* Compare medal counts for host vs non-host years
* Calculate % change in medals when hosting
* Identify countries that benefited from hosting


Visuals

* Before/after bar charts
* line chart per host country

In [17]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure, scatter, plot_date
from patsy.test_state import test_stateful_transform_wrapper
from scipy.stats import (
    ttest_ind,
    mannwhitneyu,
    ttest_rel
)

In [18]:
file_path = 'C:/Users/viole/dev/analytics/kaggle/olympics-data-analysis/data/olympic_countries_efficiency.csv'
df = pd.read_csv(file_path)
df.head()

,NOC,ISO3,Year,population,gdp_per_capita,income_group,host_country,athletes_sent,sports_participated,events_participated,female_athlete_percentage,prev_total_medals,prev_medals_per_athlete,Gold,Silver,Bronze,total_medals,medals_per_athlete
0,AFG,AFG,2004,23560654.0,221.763654,Low income,0,5,4,5,40.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
1,AFG,AFG,2008,26482622.0,381.733238,Low income,0,4,2,4,25.000000,0.0,0.000000,0.0,0.0,1.0,1.0,0.250000
2,AFG,AFG,2012,30560034.0,651.417134,Low income,0,6,4,6,16.666667,1.0,0.250000,0.0,0.0,1.0,1.0,0.166667
3,AFG,AFG,2016,34700612.0,522.082216,Low income,0,3,2,3,33.333333,1.0,0.166667,0.0,0.0,0.0,0.0,0.000000
4,ALB,ALB,1992,3247039.0,200.852220,Low income,0,7,4,8,22.222222,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


In [19]:
df.columns

Index(['NOC', 'ISO3', 'Year', 'population', 'gdp_per_capita', 'income_group',
       'host_country', 'athletes_sent', 'sports_participated',
       'events_participated', 'female_athlete_percentage', 'prev_total_medals',
       'prev_medals_per_athlete', 'Gold', 'Silver', 'Bronze', 'total_medals',
       'medals_per_athlete'],
      dtype='str')

In [20]:
# Keep only columns needed, create a copy of df
df_small = df[[
    'NOC',
    'Year',
    'host_country',
    'total_medals'
]].copy()

df_small.head()

,NOC,Year,host_country,total_medals
0,AFG,2004,0,0.0
1,AFG,2008,0,1.0
2,AFG,2012,0,1.0
3,AFG,2016,0,0.0
4,ALB,1992,0,0.0


## Host vs Non-Hosting Years

In [21]:
# Box plot to compare medals in host and non-host years
fig = px.box(
    df_small,
    x='host_country',
    y='total_medals',
    points='all',
    title='Olympic Medal Counts: Host vs Non-Host Years',
    labels={
        'host_country': 'Host Country',
        'total_medals': 'Total Medals'
    }
)

fig.update_xaxes(
    tickvals=[0, 1],
    ticktext=['Non-Host Year', 'Host Year']
)

fig.update_layout(template='plotly_dark')
fig.show()

Each point represents a single country-year observation, overlaid on box plots summarizing medal count distributions, sorted by host and non-host years. Countries hosting the Olympics had much higher summary stats than non-hosting county/years. The scatter plot on the left-hand side shows that most countries do not host and get low medal counts as expected.

In [22]:
# Bar Chart with average medals
df_host_avg = (
    df.groupby('host_country')['total_medals']
      .mean()
      .reset_index()
)

fig = px.bar(
    df_host_avg,
    x='host_country',
    y='total_medals',
    title='Average Olympic Medals: Host vs Non-Host Years',
    labels={
        'host_country': 'Host Country',
        'total_medals': 'Total Medals'
    }
)

fig.update_xaxes(
    tickvals=[0, 1],
    ticktext=['Non-Host Year', 'Host Year']
)

fig.update_layout(template='plotly_dark')
fig.show()

In [23]:
# Time-based View
fig = px.line(
    df.sort_values('Year'),
    x='Year',
    y='total_medals',
    color='host_country',
    hover_name='NOC',
    title='Medal Counts Over Time: Host vs Non-Host Years',
)

fig.update_layout(template='plotly_dark')
fig.show()

## Statistical Analysis

### Two-Sample t-test (baseline)

Compare Medal Counts:
* Group 1: host years
* Group 2: non-host years

In [24]:
# Get a series with total_medals for host countries
host_medals = df_small[
    df_small['host_country'] == 1
]['total_medals']

# Get a series with total_medals for non-host countries
non_host_medals = df_small[
    df_small['host_country'] == 0
]['total_medals']

# t_stat and p_value
t_stat, p_value = ttest_ind(
    host_medals,
    non_host_medals,
    equal_var=False
)

t_stat, p_value

(np.float64(2.2663983942084966), np.float64(0.10798720887274255))

P-value > 0.05 so not statistically significant

### Non-Parametric Alternative

Medal Counts are skewed so use Mann-Whitney U test

In [25]:
u_stat, p_value = mannwhitneyu(
    host_medals,
    non_host_medals,
    alternative='two-sided'
)

u_stat, p_value

(np.float64(915.0), np.float64(0.0017199771521243504))

P-Value < 0.05 so statistically significant

## Compare within-country

In [26]:
df_pivot = (
    df_small.pivot_table(
        index='NOC',
        columns='host_country',
        values='total_medals',
        aggfunc='mean'
    )
    .dropna()
)

df_pivot.columns = ['Non-Host', 'Host']

### Paired t-test

In [27]:
t_stat, p_value = ttest_rel(
    df_pivot['Host'],
    df_pivot['Non-Host']
)

t_stat, p_value

(np.float64(1.8374933912670544), np.float64(0.16344516413603002))

## Percent Change when Hosting

In [31]:
df_pivot

,Non-Host,Host
NOC,,
AUS,69.642857,183.0
BRA,28.857143,50.0
CAN,34.307692,23.0
CHN,90.625000,184.0


In [35]:
# Calculate % change column
df_pivot['pct_change_when_hosting'] = np.where(
    df_pivot['Non-Host'] > 0, # guard against zero
    (df_pivot['Host'] - df_pivot['Non-Host'])
    / df_pivot['Non-Host'] * 100,
    np.nan
)

df_pivot.head()

,Non-Host,Host,pct_change_when_hosting
NOC,,,
AUS,69.642857,183.0,162.769231
BRA,28.857143,50.0,73.267327
CAN,34.307692,23.0,-32.959641
CHN,90.625000,184.0,103.034483


In [36]:
# Sanity check for top values
df_pivot.sort_values('pct_change_when_hosting', ascending=False).head(10)

,Non-Host,Host,pct_change_when_hosting
NOC,,,
AUS,69.642857,183.0,162.769231
CHN,90.625000,184.0,103.034483
BRA,28.857143,50.0,73.267327
CAN,34.307692,23.0,-32.959641


In [37]:
# Distribution of % Change in Medals when Hosting
fig = px.histogram(
    df_pivot.dropna(subset=['pct_change_when_hosting']),
    x='pct_change_when_hosting',
    nbins=30,
    title='Distribution of % Change in Medals When Hosting',
    labels={
        'pct_change_when_hosting': '% Change in Medals'
    }
)

fig.update_layout(template='plotly_dark')
fig.show()

In [39]:
# Country level comparison
fig = px.bar(
    df_pivot.sort_values(
        'pct_change_when_hosting',
        ascending=False
    ),
    x=df_pivot.index,
    y='pct_change_when_hosting',
    title='% Change in Medals When Hosting by Country'
)

fig.update_layout(
    template='plotly_dark',
    xaxis={
        'categoryorder': 'total ascending',
    }
)

fig.show()

## Identify Countries that Benefited the Most from Hosting

In [40]:
# filter dataframe where Non-Host medals are >= 2
df_pivot = df_pivot[
    df_pivot['Non-Host'] >= 2
]

# Rank Countres by benefit
df_sorted = df_pivot.sort_values(
    'pct_change_when_hosting',
    ascending=False
)

df_sorted.head(10)

,Non-Host,Host,pct_change_when_hosting
NOC,,,
AUS,69.642857,183.0,162.769231
CHN,90.625000,184.0,103.034483
BRA,28.857143,50.0,73.267327
CAN,34.307692,23.0,-32.959641
